# Day 4: Hierarchical Risk Parity (HRP)

## Learning Objectives
- Understand the limitations of traditional mean-variance optimization
- Learn the Hierarchical Risk Parity algorithm by Marcos López de Prado
- Implement HRP from scratch: clustering, quasi-diagonalization, recursive bisection
- Compare HRP with traditional portfolio optimization methods

## Why HRP?

Traditional Mean-Variance Optimization (MVO) suffers from:
1. **Instability**: Small changes in inputs → large changes in weights
2. **Concentration**: Often produces extreme/concentrated portfolios
3. **Estimation error**: Covariance matrix inversion amplifies errors
4. **Singularity issues**: When N > T, covariance matrix is singular

HRP addresses these by:
- Using hierarchical clustering to group similar assets
- Avoiding matrix inversion entirely
- Producing more stable, diversified portfolios

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import squareform
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)
print("Libraries loaded successfully!")

---
## 1. Data Preparation

Let's fetch a diversified set of ETFs representing different asset classes.

In [ ]:
# Diversified ETF universe
tickers = [
    'SPY',   # S&P 500
    'QQQ',   # Nasdaq 100
    'IWM',   # Russell 2000
    'EFA',   # International Developed
    'EEM',   # Emerging Markets
    'TLT',   # Long-Term Treasuries
    'IEF',   # Intermediate Treasuries
    'LQD',   # Investment Grade Corporate Bonds
    'HYG',   # High Yield Bonds
    'GLD',   # Gold
    'VNQ',   # REITs
    'DBC',   # Commodities
]

# Download data
print("Downloading ETF data...")
data = yf.download(tickers, start='2018-01-01', end='2024-01-01', progress=False)['Adj Close']
data = data.dropna()

# Calculate returns
returns = data.pct_change().dropna()

print(f"\nData shape: {returns.shape}")
print(f"Date range: {returns.index[0].date()} to {returns.index[-1].date()}")
returns.head()

In [ ]:
# Calculate covariance and correlation matrices
cov_matrix = returns.cov() * 252  # Annualized
corr_matrix = returns.corr()

# Visualize correlation matrix
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdYlGn', center=0, ax=ax,
            vmin=-1, vmax=1)
ax.set_title('Correlation Matrix of ETF Returns', fontsize=14)
plt.tight_layout()
plt.show()

---
## 2. HRP Algorithm Overview

The HRP algorithm consists of three main steps:

### Step 1: Tree Clustering
- Convert correlation matrix to distance matrix
- Apply hierarchical clustering (agglomerative)
- Result: A dendrogram showing asset relationships

### Step 2: Quasi-Diagonalization
- Reorder covariance matrix based on clustering
- Similar assets are placed close together
- Result: Block-diagonal-like structure

### Step 3: Recursive Bisection
- Split assets into two clusters
- Allocate between clusters based on inverse variance
- Recursively repeat for sub-clusters
- Result: Final portfolio weights

---
## 3. Step 1: Tree Clustering

Convert correlation to distance and perform hierarchical clustering.

In [ ]:
def correlation_to_distance(corr):
    """
    Convert correlation matrix to distance matrix.
    
    Distance = sqrt(0.5 * (1 - correlation))
    
    Properties:
    - correlation = 1 → distance = 0
    - correlation = 0 → distance = 0.707
    - correlation = -1 → distance = 1
    """
    distance = np.sqrt(0.5 * (1 - corr))
    return distance

# Calculate distance matrix
dist_matrix = correlation_to_distance(corr_matrix)

print("Distance Matrix (sample):")
print(dist_matrix.iloc[:5, :5].round(3))

In [ ]:
def perform_clustering(dist_matrix, method='single'):
    """
    Perform hierarchical clustering on distance matrix.
    
    Parameters:
    -----------
    dist_matrix : DataFrame
        Distance matrix
    method : str
        Linkage method: 'single', 'complete', 'average', 'ward'
    
    Returns:
    --------
    linkage_matrix : ndarray
        Hierarchical clustering encoded as linkage matrix
    """
    # Convert to condensed distance matrix (upper triangular)
    condensed_dist = squareform(dist_matrix.values)
    
    # Perform hierarchical clustering
    linkage_matrix = linkage(condensed_dist, method=method)
    
    return linkage_matrix

# Perform clustering
linkage_mat = perform_clustering(dist_matrix, method='single')

print("Linkage matrix shape:", linkage_mat.shape)
print("\nLinkage matrix format: [cluster1, cluster2, distance, n_members]")

In [ ]:
# Visualize dendrogram
fig, ax = plt.subplots(figsize=(14, 6))

dendrogram(
    linkage_mat,
    labels=returns.columns.tolist(),
    leaf_rotation=45,
    ax=ax
)

ax.set_title('Hierarchical Clustering Dendrogram', fontsize=14)
ax.set_xlabel('Assets')
ax.set_ylabel('Distance')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Lower merge height = more similar assets")
print("- Clusters: Equities (SPY, QQQ, IWM), Bonds (TLT, IEF), etc.")

---
## 4. Step 2: Quasi-Diagonalization

Reorder assets so that similar assets are adjacent.

In [ ]:
def quasi_diagonalize(linkage_mat, n_assets):
    """
    Reorder assets based on hierarchical clustering.
    
    Returns the order of assets that quasi-diagonalizes the covariance matrix.
    """
    # Get the order of leaves from dendrogram
    sorted_indices = leaves_list(linkage_mat)
    return sorted_indices

# Get sorted order
sorted_idx = quasi_diagonalize(linkage_mat, len(returns.columns))
sorted_tickers = [returns.columns[i] for i in sorted_idx]

print("Original order:", list(returns.columns))
print("\nClustered order:", sorted_tickers)

In [ ]:
# Compare original vs quasi-diagonalized covariance matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original order
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, ax=axes[0], vmin=-1, vmax=1,
            annot_kws={'size': 8})
axes[0].set_title('Original Correlation Matrix', fontsize=12)

# Quasi-diagonalized order
corr_sorted = corr_matrix.iloc[sorted_idx, sorted_idx]
sns.heatmap(corr_sorted, annot=True, fmt='.2f', cmap='RdYlGn',
            center=0, ax=axes[1], vmin=-1, vmax=1,
            annot_kws={'size': 8})
axes[1].set_title('Quasi-Diagonalized Correlation Matrix', fontsize=12)

plt.tight_layout()
plt.show()

print("Notice how similar assets (high correlation) are now adjacent!")

---
## 5. Step 3: Recursive Bisection

The core allocation algorithm that splits clusters and allocates based on variance.

In [ ]:
def get_cluster_variance(cov, assets):
    """
    Calculate the variance of a cluster using inverse-variance weights.
    
    Parameters:
    -----------
    cov : DataFrame
        Covariance matrix
    assets : list
        List of asset names in the cluster
    
    Returns:
    --------
    cluster_var : float
        Variance of the cluster portfolio
    """
    # Extract sub-covariance matrix
    cov_slice = cov.loc[assets, assets]
    
    # Inverse variance weights within cluster
    variances = np.diag(cov_slice)
    inv_var = 1.0 / variances
    weights = inv_var / inv_var.sum()
    
    # Portfolio variance: w' * Σ * w
    cluster_var = np.dot(weights, np.dot(cov_slice, weights))
    
    return cluster_var

# Test
test_assets = ['SPY', 'QQQ', 'IWM']
test_var = get_cluster_variance(cov_matrix, test_assets)
print(f"Cluster variance for {test_assets}: {test_var:.6f}")

In [ ]:
def recursive_bisection(cov, sorted_assets):
    """
    Perform recursive bisection to calculate HRP weights.
    
    Parameters:
    -----------
    cov : DataFrame
        Covariance matrix
    sorted_assets : list
        Assets in quasi-diagonalized order
    
    Returns:
    --------
    weights : Series
        HRP portfolio weights
    """
    # Initialize weights
    weights = pd.Series(1.0, index=sorted_assets)
    
    # List of clusters to process
    clusters = [sorted_assets]
    
    while len(clusters) > 0:
        # Split each cluster
        new_clusters = []
        
        for cluster in clusters:
            if len(cluster) > 1:
                # Split cluster in half
                mid = len(cluster) // 2
                cluster_1 = cluster[:mid]
                cluster_2 = cluster[mid:]
                
                # Calculate cluster variances
                var_1 = get_cluster_variance(cov, cluster_1)
                var_2 = get_cluster_variance(cov, cluster_2)
                
                # Allocation factor (inverse variance weighting)
                alpha = 1 - var_1 / (var_1 + var_2)
                
                # Update weights
                weights[cluster_1] *= alpha
                weights[cluster_2] *= (1 - alpha)
                
                # Add sub-clusters for further processing
                if len(cluster_1) > 1:
                    new_clusters.append(cluster_1)
                if len(cluster_2) > 1:
                    new_clusters.append(cluster_2)
        
        clusters = new_clusters
    
    return weights

# Calculate HRP weights
hrp_weights = recursive_bisection(cov_matrix, sorted_tickers)

# Reorder to original ticker order
hrp_weights = hrp_weights.reindex(returns.columns)

print("HRP Weights:")
print(hrp_weights.sort_values(ascending=False).round(4))
print(f"\nSum of weights: {hrp_weights.sum():.6f}")

---
## 6. Complete HRP Implementation

Let's wrap everything into a clean class.

In [ ]:
class HierarchicalRiskParity:
    """
    Hierarchical Risk Parity (HRP) Portfolio Optimization.
    
    Based on: López de Prado, M. (2016). Building Diversified Portfolios 
    that Outperform Out-of-Sample.
    """
    
    def __init__(self, linkage_method='single'):
        """
        Parameters:
        -----------
        linkage_method : str
            Hierarchical clustering linkage method
            Options: 'single', 'complete', 'average', 'ward'
        """
        self.linkage_method = linkage_method
        self.weights = None
        self.linkage_matrix = None
        self.sorted_assets = None
        
    def _correlation_to_distance(self, corr):
        """Convert correlation to distance matrix."""
        return np.sqrt(0.5 * (1 - corr))
    
    def _get_cluster_variance(self, cov, assets):
        """Calculate cluster variance using inverse-variance weights."""
        cov_slice = cov.loc[assets, assets]
        variances = np.diag(cov_slice)
        inv_var = 1.0 / variances
        w = inv_var / inv_var.sum()
        return np.dot(w, np.dot(cov_slice, w))
    
    def _recursive_bisection(self, cov, sorted_assets):
        """Perform recursive bisection for weight allocation."""
        weights = pd.Series(1.0, index=sorted_assets)
        clusters = [sorted_assets]
        
        while clusters:
            new_clusters = []
            for cluster in clusters:
                if len(cluster) > 1:
                    mid = len(cluster) // 2
                    c1, c2 = cluster[:mid], cluster[mid:]
                    
                    var1 = self._get_cluster_variance(cov, c1)
                    var2 = self._get_cluster_variance(cov, c2)
                    
                    alpha = 1 - var1 / (var1 + var2)
                    
                    weights[c1] *= alpha
                    weights[c2] *= (1 - alpha)
                    
                    if len(c1) > 1: new_clusters.append(c1)
                    if len(c2) > 1: new_clusters.append(c2)
            
            clusters = new_clusters
        
        return weights
    
    def fit(self, returns):
        """
        Fit HRP model to returns data.
        
        Parameters:
        -----------
        returns : DataFrame
            Asset returns (T x N)
        
        Returns:
        --------
        self
        """
        # Calculate covariance and correlation
        cov = returns.cov()
        corr = returns.corr()
        
        # Step 1: Tree clustering
        dist = self._correlation_to_distance(corr)
        condensed_dist = squareform(dist.values)
        self.linkage_matrix = linkage(condensed_dist, method=self.linkage_method)
        
        # Step 2: Quasi-diagonalization
        sorted_idx = leaves_list(self.linkage_matrix)
        self.sorted_assets = [returns.columns[i] for i in sorted_idx]
        
        # Step 3: Recursive bisection
        weights = self._recursive_bisection(cov, self.sorted_assets)
        
        # Reorder to original order
        self.weights = weights.reindex(returns.columns)
        
        return self
    
    def get_weights(self):
        """Return portfolio weights."""
        return self.weights
    
    def plot_dendrogram(self, figsize=(12, 5)):
        """Plot the hierarchical clustering dendrogram."""
        if self.linkage_matrix is None:
            raise ValueError("Must call fit() first")
            
        fig, ax = plt.subplots(figsize=figsize)
        dendrogram(self.linkage_matrix, labels=self.sorted_assets,
                  leaf_rotation=45, ax=ax)
        ax.set_title('HRP Dendrogram')
        ax.set_ylabel('Distance')
        plt.tight_layout()
        return fig
    
    def plot_weights(self, figsize=(10, 5)):
        """Plot portfolio weights."""
        if self.weights is None:
            raise ValueError("Must call fit() first")
            
        fig, ax = plt.subplots(figsize=figsize)
        self.weights.sort_values().plot(kind='barh', ax=ax, color='steelblue')
        ax.set_xlabel('Weight')
        ax.set_title('HRP Portfolio Weights')
        ax.axvline(x=1/len(self.weights), color='red', linestyle='--', 
                   label='Equal Weight')
        ax.legend()
        plt.tight_layout()
        return fig

In [ ]:
# Use the HRP class
hrp = HierarchicalRiskParity(linkage_method='single')
hrp.fit(returns)

print("HRP Portfolio Weights:")
print(hrp.get_weights().round(4))

# Plot
hrp.plot_weights()
plt.show()

---
## 7. Comparison with Other Methods

Let's compare HRP with:
1. Equal Weight (1/N)
2. Inverse Variance
3. Mean-Variance Optimization (Minimum Variance)

In [ ]:
def equal_weight(returns):
    """Equal weight portfolio."""
    n = len(returns.columns)
    return pd.Series(1/n, index=returns.columns)

def inverse_variance(returns):
    """Inverse variance (risk parity on individual assets)."""
    cov = returns.cov()
    var = np.diag(cov)
    inv_var = 1 / var
    weights = inv_var / inv_var.sum()
    return pd.Series(weights, index=returns.columns)

def min_variance(returns):
    """Minimum variance portfolio (analytical solution)."""
    cov = returns.cov()
    n = len(returns.columns)
    
    # Analytical solution: w = Σ^(-1) * 1 / (1' * Σ^(-1) * 1)
    try:
        cov_inv = np.linalg.inv(cov)
        ones = np.ones(n)
        weights = cov_inv @ ones / (ones @ cov_inv @ ones)
        return pd.Series(weights, index=returns.columns)
    except:
        # If singular, use pseudo-inverse
        cov_inv = np.linalg.pinv(cov)
        ones = np.ones(n)
        weights = cov_inv @ ones / (ones @ cov_inv @ ones)
        return pd.Series(weights, index=returns.columns)

# Calculate all weights
weights_dict = {
    'Equal Weight': equal_weight(returns),
    'Inverse Variance': inverse_variance(returns),
    'Min Variance': min_variance(returns),
    'HRP': hrp.get_weights()
}

# Create comparison DataFrame
weights_df = pd.DataFrame(weights_dict)
print("Portfolio Weights Comparison:")
print(weights_df.round(4))

In [ ]:
# Visualize weight comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(returns.columns))
width = 0.2

for i, (name, weights) in enumerate(weights_dict.items()):
    ax.bar(x + i*width, weights.values, width, label=name, alpha=0.8)

ax.set_xlabel('Asset')
ax.set_ylabel('Weight')
ax.set_title('Portfolio Weights: Different Methods')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(returns.columns, rotation=45)
ax.legend()
ax.axhline(y=0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
def portfolio_metrics(returns, weights):
    """
    Calculate portfolio performance metrics.
    """
    # Portfolio returns
    port_ret = (returns * weights).sum(axis=1)
    
    # Annualized metrics
    ann_ret = port_ret.mean() * 252
    ann_vol = port_ret.std() * np.sqrt(252)
    sharpe = ann_ret / ann_vol
    
    # Max drawdown
    cum_ret = (1 + port_ret).cumprod()
    rolling_max = cum_ret.expanding().max()
    drawdown = (cum_ret - rolling_max) / rolling_max
    max_dd = drawdown.min()
    
    # Concentration (Herfindahl index)
    hhi = (weights ** 2).sum()
    effective_n = 1 / hhi  # Effective number of assets
    
    return {
        'Annual Return': f"{ann_ret:.2%}",
        'Annual Volatility': f"{ann_vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.3f}",
        'Max Drawdown': f"{max_dd:.2%}",
        'Effective N': f"{effective_n:.1f}",
        'HHI': f"{hhi:.4f}"
    }

# Calculate metrics for all portfolios
metrics = {}
for name, weights in weights_dict.items():
    metrics[name] = portfolio_metrics(returns, weights)

metrics_df = pd.DataFrame(metrics)
print("\nPortfolio Performance Comparison (In-Sample):")
print(metrics_df)

In [ ]:
# Plot cumulative returns
fig, ax = plt.subplots(figsize=(12, 6))

for name, weights in weights_dict.items():
    port_ret = (returns * weights).sum(axis=1)
    cum_ret = (1 + port_ret).cumprod()
    ax.plot(cum_ret.index, cum_ret.values, label=name, linewidth=1.5)

ax.set_title('Cumulative Returns: Different Portfolio Methods')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. Out-of-Sample Testing with Rolling Windows

The real test of HRP is out-of-sample performance.

In [ ]:
def rolling_backtest(returns, window=252, rebalance_freq=21):
    """
    Perform rolling window backtest for different portfolio methods.
    
    Parameters:
    -----------
    returns : DataFrame
        Asset returns
    window : int
        Lookback window for estimation (days)
    rebalance_freq : int
        Rebalancing frequency (days)
    """
    results = {name: [] for name in ['Equal Weight', 'Inverse Variance', 'Min Variance', 'HRP']}
    dates = []
    
    for i in range(window, len(returns), rebalance_freq):
        # Estimation window
        train = returns.iloc[i-window:i]
        
        # Test period (until next rebalance)
        test_end = min(i + rebalance_freq, len(returns))
        test = returns.iloc[i:test_end]
        
        if len(test) == 0:
            continue
            
        # Calculate weights
        w_eq = equal_weight(train)
        w_iv = inverse_variance(train)
        w_mv = min_variance(train)
        
        hrp_model = HierarchicalRiskParity()
        hrp_model.fit(train)
        w_hrp = hrp_model.get_weights()
        
        # Calculate test period returns
        for name, w in [('Equal Weight', w_eq), ('Inverse Variance', w_iv), 
                        ('Min Variance', w_mv), ('HRP', w_hrp)]:
            port_ret = (test * w).sum(axis=1)
            results[name].extend(port_ret.values)
        
        dates.extend(test.index)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results, index=dates)
    return results_df

print("Running rolling backtest (this may take a moment)...")
backtest_results = rolling_backtest(returns, window=252, rebalance_freq=21)
print(f"Backtest period: {backtest_results.index[0].date()} to {backtest_results.index[-1].date()}")

In [ ]:
# Calculate out-of-sample metrics
oos_metrics = {}

for col in backtest_results.columns:
    ret = backtest_results[col]
    
    ann_ret = ret.mean() * 252
    ann_vol = ret.std() * np.sqrt(252)
    sharpe = ann_ret / ann_vol
    
    cum_ret = (1 + ret).cumprod()
    rolling_max = cum_ret.expanding().max()
    max_dd = ((cum_ret - rolling_max) / rolling_max).min()
    
    oos_metrics[col] = {
        'Annual Return': f"{ann_ret:.2%}",
        'Annual Volatility': f"{ann_vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.3f}",
        'Max Drawdown': f"{max_dd:.2%}"
    }

print("\nOut-of-Sample Performance:")
print(pd.DataFrame(oos_metrics))

In [ ]:
# Plot out-of-sample cumulative returns
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Cumulative returns
cum_returns = (1 + backtest_results).cumprod()
for col in cum_returns.columns:
    axes[0].plot(cum_returns.index, cum_returns[col], label=col, linewidth=1.5)

axes[0].set_title('Out-of-Sample Cumulative Returns')
axes[0].set_ylabel('Cumulative Return')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdowns
for col in cum_returns.columns:
    rolling_max = cum_returns[col].expanding().max()
    drawdown = (cum_returns[col] - rolling_max) / rolling_max
    axes[1].fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, label=col)

axes[1].set_title('Drawdowns')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Drawdown')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 9. Stability Analysis

One key advantage of HRP is weight stability over time.

In [ ]:
def calculate_weight_turnover(returns, method_func, window=252, rebalance_freq=21):
    """
    Calculate weight turnover over time.
    """
    prev_weights = None
    turnovers = []
    dates = []
    
    for i in range(window, len(returns), rebalance_freq):
        train = returns.iloc[i-window:i]
        
        if method_func == 'hrp':
            model = HierarchicalRiskParity()
            model.fit(train)
            weights = model.get_weights()
        else:
            weights = method_func(train)
        
        if prev_weights is not None:
            turnover = np.abs(weights - prev_weights).sum() / 2
            turnovers.append(turnover)
            dates.append(returns.index[i])
        
        prev_weights = weights
    
    return pd.Series(turnovers, index=dates)

# Calculate turnover for each method
turnover_eq = calculate_weight_turnover(returns, equal_weight)
turnover_iv = calculate_weight_turnover(returns, inverse_variance)
turnover_mv = calculate_weight_turnover(returns, min_variance)
turnover_hrp = calculate_weight_turnover(returns, 'hrp')

# Summary
turnover_summary = pd.DataFrame({
    'Equal Weight': [turnover_eq.mean(), turnover_eq.std()],
    'Inverse Variance': [turnover_iv.mean(), turnover_iv.std()],
    'Min Variance': [turnover_mv.mean(), turnover_mv.std()],
    'HRP': [turnover_hrp.mean(), turnover_hrp.std()]
}, index=['Mean Turnover', 'Std Turnover'])

print("Weight Turnover Analysis (lower = more stable):")
print(turnover_summary.round(4))

In [ ]:
# Plot turnover over time
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(turnover_iv.index, turnover_iv.values, label='Inverse Variance', alpha=0.7)
ax.plot(turnover_mv.index, turnover_mv.values, label='Min Variance', alpha=0.7)
ax.plot(turnover_hrp.index, turnover_hrp.values, label='HRP', alpha=0.7)

ax.set_title('Portfolio Turnover Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Turnover (fraction of portfolio)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: HRP typically shows lower and more stable turnover than MVO.")

---
## 10. Interview Questions & Key Takeaways

### Common Interview Questions:

**Q1: What problem does HRP solve?**
> HRP addresses the instability and concentration issues of mean-variance optimization by using hierarchical clustering and avoiding matrix inversion.

**Q2: Explain the three steps of HRP.**
> 1. **Tree Clustering**: Convert correlation to distance, perform hierarchical clustering
> 2. **Quasi-Diagonalization**: Reorder assets so similar ones are adjacent
> 3. **Recursive Bisection**: Split clusters and allocate using inverse variance weights

**Q3: Why does HRP avoid matrix inversion?**
> Matrix inversion amplifies estimation errors and can be numerically unstable, especially when N > T or assets are highly correlated.

**Q4: How does HRP compare to Risk Parity?**
> Traditional Risk Parity ignores correlations (or uses inverse covariance). HRP accounts for the hierarchical structure of correlations while maintaining stability.

**Q5: What are limitations of HRP?**
> - No explicit return forecasts (like min variance)
> - Results depend on linkage method choice
> - May underperform when correlations are stable and known

### Key Takeaways:
1. HRP is a machine learning approach to portfolio construction
2. It produces more stable, diversified portfolios than MVO
3. Works well when covariance estimates are noisy
4. Lower turnover = lower transaction costs
5. Particularly useful for large universes where N > T

In [ ]:
# Summary
print("=" * 60)
print("DAY 4 COMPLETE: Hierarchical Risk Parity")
print("=" * 60)
print("""
Key Concepts Covered:
✓ Why traditional MVO fails
✓ Correlation-to-distance transformation
✓ Hierarchical clustering for asset grouping
✓ Quasi-diagonalization of covariance matrix
✓ Recursive bisection for weight allocation
✓ Complete HRP implementation
✓ Comparison with Equal Weight, IVP, Min Variance
✓ Out-of-sample rolling backtest
✓ Turnover/stability analysis

Tomorrow: Black-Litterman Model
""")